# Семинар: Градиентный спуск. Задачи 

In [ ]:
#from typing import Iterable, List

import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline

## Часть 1. Градиентный спуск

Функционал ошибки, который мы применяем в задаче регрессии — Mean Squared Error:

$$
Q(w, X, y) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\langle x_i, w \rangle - y_i)^2
$$

где $x_i$ — это $i$-ый объект датасета, $y_i$ — правильный ответ для $i$-го объекта, а $w$ — веса нашей линейной модели.

Можно показать, что для линейной модели, функционал ошибки можно записать в матричном виде следующим образом:
$$
Q(w, X, y) =\frac{1}{l} (y - Xw)^T(y-Xw)
$$
или
$$
Q(w, X, y) = \frac{1}{l} || Xw - y ||^2
$$

где $X$ — это матрица объекты-признаки, а $y$ — вектор правильных ответов

Для того чтобы воспользоваться методом градиентного спуска, нам нужно посчитать градиент нашего функционала. Для MSE он будет выглядеть так:

$$
\nabla_w Q(w, X, y) = \frac{2}{l} X^T(Xw-y)
$$

Ниже приведён базовый класс `BaseLoss`, который мы будем использовать для реализации всех наших функционалов ошибки (= функций потерь = лоссов). Менять его не нужно. У него есть два абстрактных метода:
1. Метод `calc_loss`, который будет принимать на вход объекты `x`, правильные ответы `y` и веса `w` и вычислять значения функционала ошибки
2. Метод `calc_grad`, который будет принимать на вход объекты `x`, правильные ответы `y` и веса `w` и вычислять значения градиента (вектор)

In [ ]:
import abc


class BaseLoss(abc.ABC):
    """Базовый класс лосса"""

    @abc.abstractmethod
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        raise NotImplementedError

    @abc.abstractmethod
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        raise NotImplementedError

Реализация этого абстрактного класса: Mean Squared Error лосс.

**Задание 1.1:** __Реализуйте класс `MSELoss`__

Он должен вычислять значение функционала ошибки (лосс) $
Q(w, X, y)$ и его градиент $\nabla_w Q(w, X, y)$ по формулам (выше)

In [ ]:
class MSELoss(BaseLoss):
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        # -- YOUR CODE HERE --
        # Вычислите значение функции потерь при помощи X, y и w и верните его   
  

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        # -- YOUR CODE HERE --
        # Вычислите значение вектора градиента при помощи X, y и w и верните его


Теперь мы можем создать объект `MSELoss` и при помощи него вычислять значение нашего функционала ошибки и градиент:

In [ ]:
# Создадим объект лосса
loss = MSELoss()

# Создадим датасет
X = np.arange(200).reshape(20, 10)
y = np.arange(20)

# Создадим вектор весов
w = np.arange(10)

#print(X)
#print(y)
#print(w)

# Выведем значение лосса и градиента на этом датасете с этим вектором весов
print(loss.calc_loss(X, y, w))
print(loss.calc_grad(X, y, w))

# Проверка, что методы реализованы правильно
assert loss.calc_loss(X, y, w) == 27410283.5, "Метод calc_loss реализован неверно"
assert np.allclose(
    loss.calc_grad(X, y, w),
    np.array(
        [
            1163180.0,
            1172281.0,
            1181382.0,
            1190483.0,
            1199584.0,
            1208685.0,
            1217786.0,
            1226887.0,
            1235988.0,
            1245089.0,
        ]
    ),
), "Метод calc_grad реализован неверно"
print("Всё верно!")

Для вычисления градиента все готово, реализуем градиентный спуск. Напомним, что формула для одной итерации градиентного спуска выглядит следующим 
образом:

$$
w^t = w^{t-1} - \eta \nabla_{w} Q(w^{t-1}, X, y)
$$

Где $w^t$ — значение вектора весов на $t$-ой итерации, а $\eta$ — параметр learning rate, отвечающий за размер шага.

**Задание 1.2:** __Реализуйте функцию `gradient_descent`__

Функция должна принимать на вход начальное значение весов линейной модели `w_init`, матрицу объектов-признаков `X`, 
вектор правильных ответов `y`, объект функции потерь `loss`, размер шага `lr` и количество итераций `n_iterations`.

Функция должна реализовывать цикл, в котором осуществляется шаг градиентного спуска (градиенты берутся из `loss` посредством вызова метода `calc_grad`) по формуле выше и возвращать 
траекторию спуска (список из новых значений весов на каждом шаге)

In [2]:
def gradient_descent(
    w_init: np.ndarray,
    X: np.ndarray,
    y: np.ndarray,
    loss: BaseLoss,
    lr: float,
    n_iterations: int = 100000,
) -> List[np.ndarray]:
    """
    Функция градиентного спуска
    :param w_init: np.ndarray размера (n_feratures,) -- начальное значение вектора весов
    :param X: np.ndarray размера (n_objects, n_features) -- матрица объекты-признаки
    :param y: np.ndarray размера (n_objects,) -- вектор правильных ответов
    :param loss: Объект подкласса BaseLoss, который умеет считать градиенты при помощи loss.calc_grad(X, y, w)
    :param lr: float -- параметр величины шага, на который нужно домножать градиент
    :param n_iterations: int -- сколько итераций делать
    :return: Список из n_iterations объектов np.ndarray размера (n_features,) -- история весов на каждом шаге
    """
    # -- YOUR CODE HERE --


NameError: name 'np' is not defined

Теперь создадим синтетический датасет и функцию, которая будет рисовать траекторию градиентного спуска по истории.  
(Если не оговорено иное, то в задачах используются указанные параметры `n_features,
n_objects, batch_size, num_steps`). 


In [ ]:
# Создаём датасет из двух переменных и реального вектора зависимости w_true

np.random.seed(1337)

n_features = 2
n_objects = 300
batch_size = 10
num_steps = 43

w_true = np.random.normal(size=(n_features,))

X = np.random.uniform(-5, 5, (n_objects, n_features))
X *= (np.arange(n_features) * 2 + 1)[np.newaxis, :]
y = X.dot(w_true) + np.random.normal(0, 1, (n_objects))
w_init = np.random.uniform(-2, 2, (n_features))

print(X.shape)
print(y.shape)

In [ ]:
loss = MSELoss()
w_list = gradient_descent(w_init, X, y, loss, 0.01, num_steps)
print(loss.calc_loss(X, y, w_list[0]))
print(loss.calc_loss(X, y, w_list[-1]))

In [ ]:
w_init

In [ ]:
w_list

In [ ]:
def plot_gd(w_list: Iterable, X: np.ndarray, y: np.ndarray, loss: BaseLoss):
    """
    Функция для отрисовки траектории градиентного спуска
    :param w_list: Список из объектов np.ndarray размера (n_features,) -- история весов на каждом шаге
    :param X: np.ndarray размера (n_objects, n_features) -- матрица объекты-признаки
    :param y: np.ndarray размера (n_objects,) -- вектор правильных ответов
    :param loss: Объект подкласса BaseLoss, который умеет считать лосс при помощи loss.calc_loss(X, y, w)
    """
    w_list = np.array(w_list)
    meshgrid_space = np.linspace(-2, 2, 100)
    A, B = np.meshgrid(meshgrid_space, meshgrid_space)

    levels = np.empty_like(A)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            w_tmp = np.array([A[i, j], B[i, j]])
            levels[i, j] = loss.calc_loss(X, y, w_tmp)

    plt.figure(figsize=(15, 6))
    plt.title("GD trajectory")
    plt.xlabel(r"$w_1$")
    plt.ylabel(r"$w_2$")
    plt.xlim(w_list[:, 0].min() - 0.1, w_list[:, 0].max() + 0.1)
    plt.ylim(w_list[:, 1].min() - 0.1, w_list[:, 1].max() + 0.1)
    plt.gca().set_aspect("equal")

    # visualize the level set
    CS = plt.contour(
        A, B, levels, levels=np.logspace(0, 1, num=20), cmap=plt.cm.rainbow_r
    )
    CB = plt.colorbar(CS, shrink=0.8, extend="both")

    # visualize trajectory
    plt.scatter(w_list[:, 0], w_list[:, 1])
    plt.plot(w_list[:, 0], w_list[:, 1])

    plt.show()

**Задание 1.3:** __При помощи функций `gradient_descent` и  `plot_gd` нарисуйте траекторию градиентного спуска для разных значений длины шага (параметра `lr`)__. Используйте не менее четырёх разных значений для `lr`. Для каждой длины шага вычисляйтете значение функционала ошибки при помощи метода `calc_loss` на первой и последней итерациях градиентного спуска.

Сделайте и опишите свои выводы о том, как параметр `lr` влияет на поведение градиентного спуска

Подсказки:
* Функция `gradient_descent` возвращает историю весов, которую нужно подать в функцию `plot_gd`
* Хорошие значения для `lr` могут лежать в промежутке от 0.0001 до 0.1

In [ ]:
# -- YOUR CODE HERE --

Теперь реализуем стохастический градиентный спуск

**Задание 1.4:** __Реализуйте функцию `stochastic_gradient_descent`__

Функция должна принимать все те же параметры, что и функция `gradient_descent`, но ещё параметр `batch_size`, отвечающий за размер батча. 

Функция должна как и раньше реализовывать цикл, в котором происходит шаг градиентного спуска, но на каждом шаге считать градиент не по всей выборке `X`, а только по случайно выбранной части.

Подсказка: для выбора случайной части можно использовать [`np.random.choice`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) с правильным параметром `size`, чтобы выбрать случайные индексы, а потом проиндексировать получившимся массивом массив `X`:
```
batch_indices = np.random.choice(X.shape[0], size=batch_size, replace=False)
batch = X[batch_indices]
```

(*здесь np.random.choice генерирует из np.arange(X.shape[0]), длина size,
replace=False - без повторения*)

In [ ]:
X.shape[0]

In [ ]:
def stochastic_gradient_descent(
    w_init: np.ndarray,
    X: np.ndarray,
    y: np.ndarray,
    loss: BaseLoss,
    lr: float,
    batch_size: int,
    n_iterations: int = 1000,
) -> List[np.ndarray]:
    """
    Функция градиентного спуска
    :param w_init: np.ndarray размера (n_feratures,) -- начальное значение вектора весов
    :param X: np.ndarray размера (n_objects, n_features) -- матрица объекты-признаки
    :param y: np.ndarray размера (n_objects,) -- вектор правильных ответов
    :param loss: Объект подкласса BaseLoss, который умеет считать градиенты при помощи loss.calc_grad(X, y, w)
    :param lr: float -- параметр величины шага, на который нужно домножать градиент
    :param batch_size: int -- размер подвыборки, которую нужно семплировать на каждом шаге
    :param n_iterations: int -- сколько итераций делать
    :return: Список из n_iterations объектов np.ndarray размера (n_features,) -- история весов на каждом шаге
    """
    # -- YOUR CODE HERE --
    


**Задание 1.5:** __При помощи функций `stochastic_gradient_descent` и  `plot_gd` нарисуйте траекторию градиентного спуска для разных значений длины шага (параметра `lr`) и размера подвыборки (параметра `batch_size`)__. Используйте не менее трех разных значений для `lr` и `batch_size`. (Для каждых длины шага и размера подвыборки вычисляйтете значение функционала ошибки при помощи метода `calc_loss` на первой и последней итерациях стохастического градиентного спуска).

Сделайте и опишите свои выводы о том, как параметры  `lr` и `batch_size` влияют на поведение стохастического градиентного спуска. Как отличается поведение стохастического градиентного спуска от обычного?

Обратите внимание, что в нашем датасете всего 300 объектов, так что `batch_size` больше этого числа не будет иметь смысла.

In [ ]:
# -- YOUR CODE HERE --

Можно заметить, что поведение градиентного спуска, особенно его стохастической версии, очень сильно зависит от размера шага. 

Как правило, в начале спуска мы хотим делать большие шаги, чтобы поскорее подойти поближе к минимуму, а позже мы уже хотим делать шаги маленькие, чтобы более точнее этого минимума достичь и не "перепрыгнуть" его. 

Чтобы достичь такого поведения мы можем постепенно уменьшать длину шага с увеличением номера итерации. Сделать это можно, например, вычисляя на каждой итерации длину шага по следующей формуле:

$$
    \eta_t
    =
    \lambda
    \left(
        \frac{s_0}{s_0 + t}
    \right)^p
$$

где $\eta_t$ — длина шага на итерации $t$, $\lambda$ — начальная длина шага (параметр `lr` у нас), $s_0$ и $p$ — настраиваемые параметры.

**Задание 1.6:** __Реализуйте функцию `stochastic_gradient_descent` с затухающим шагом по формуле выше__. Параметр $s_0$ возьмите равным 1. Параметр $p$ возьмите из нового аргумента функции `p`.

In [ ]:
def stochastic_gradient_descent(
    w_init: np.ndarray,
    X: np.ndarray,
    y: np.ndarray,
    loss: BaseLoss,
    lr: float,
    batch_size: int,
    p: float,
    n_iterations: int = 1000,
) -> List[np.ndarray]:
    """
    Функция градиентного спуска
    :param w_init: np.ndarray размера (n_feratures,) -- начальное значение вектора весов
    :param X: np.ndarray размера (n_objects, n_features) -- матрица объекты-признаки
    :param y: np.ndarray размера (n_objects,) -- вектор правильных ответов
    :param loss: Объект подкласса BaseLoss, который умеет считать градиенты при помощи loss.calc_grad(X, y, w)
    :param lr: float -- параметр величины шага, на который нужно домножать градиент
    :param batch_size: int -- размер подвыборки, которую нужно семплировать на каждом шаге
    :param p: float -- значение степени в формуле затухания длины шага
    :param n_iterations: int -- сколько итераций делать
    :return: Список из n_iterations объектов np.ndarray размера (n_features,) -- история весов на каждом шаге
    """
    # -- YOUR CODE HERE --
    


**Задание 1.7:** __При помощи новой функции `stochastic_gradient_descent` и функции `plot_gd` нарисуйте траекторию градиентного спуска для разных значений параметра `p`__. Используйте не менее четырёх разных значений для `p`. Хорошими могут быть значения, лежащие в промежутке от 0.1 до 1.
Параметр `lr` возьмите равным 0.01, а параметр `batch_size` равным 10. (Для каждого значения параметра `p` вычисляйте значение функционала ошибки при помощи метода `calc_loss` на первой и последней итерациях стохастического градиентного спуска).

Сделайте и опишите свои выводы о том, как параметр `p` влияет на поведение стохастического градиентного спуска

In [ ]:
# -- YOUR CODE HERE --

**Задание 1.8:** __Сравните сходимость обычного градиентного спуска и его стохастической версии__:
Нарисуйте график зависимости значения функционала ошибки (лосса) (его можно посчитать при помощи метода `calc_loss`, используя $x$ и $y$ из датасета и $w$ с соответствующей итерации) от номера итерации для траекторий, полученных при помощи обычного и стохастического градиентного спуска __с одинаковыми параметрами__. В SGD параметр `batch_size` возьмите равным 10, `p=0`.


In [ ]:
print(n_features)
print(n_objects)
print(batch_size)
print(num_steps)

In [ ]:
# -- YOUR CODE HERE --

## Часть 2. Линейная регрессия 

In [ ]:
from typing import Iterable, List

Создадим класс для линейной регрессии. Он будет использовать интерфейс, знакомый нам из библиотеки `sklearn`.

В методе `fit` мы будем подбирать веса `w` при помощи градиентного спуска нашим методом `gradient_descent`.

В методе `predict` мы будем применять нашу регрессию к датасету, 

**Задание 2.1:** Допишите код в методах `fit` и `predict` класса `LinearRegression_1`

В методе `fit` вам нужно инициализировать веса `w` (например, из члучайного распределения), применить наш `gradient_descent` и сохранить последние веса `w` из траектории.

В методе `predict` вам нужно применить линейную регрессию и вернуть вектор ответов.

Обратите внимание, что объект лосса (функционала ошибки) передаётся в момент инициализации и хранится в `self.loss`. Его нужно использовать в `fit` для `gradient_descent` (например, с `n_iterations: int = 1000`).

In [ ]:
class LinearRegression_1:
    def __init__(self, loss: BaseLoss, lr: float = 0.01) -> None: 
        #loss - функционал ошибки
        #lr - градиентный шаг
        self.loss = loss
        self.lr = lr

    def fit(self, X: np.ndarray, y: np.ndarray) -> "LinearRegression_1":
        X = np.asarray(X)
        y = np.asarray(y)
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        # -- YOUR CODE HERE --  
        
        return self      

    
    def predict(self, X: np.ndarray) -> np.ndarray:
        # Проверяем, что регрессия обучена, то есть, что был вызван fit и в нём был установлен атрибут self.w
        assert hasattr(self, "w"), "Linear regression must be fitted first"
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        # -- YOUR CODE HERE --

Класс линейной регрессии создан. Более того, мы можем управлять тем, какой функционал ошибки мы оптимизируем, просто передавая разные классы в параметр `loss` при инициализации. 


Будем применять нашу регрессию к реальному датасету. Загрузим датасет с машинами (см. семинар 5_sem-sklearn-knn.ipynb):

In [ ]:
import pandas as pd

X_raw = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
    header=None,      #в исх. таблице нет названий колонок
    na_values=["?"],  #если ?, то NaN
)

X_raw.head()
X_raw = X_raw[~X_raw[25].isna()].reset_index(drop=True)

In [ ]:
y = X_raw[25]
X_raw = X_raw.drop(25, axis=1)

**Задание 2.2:** Обработайте датасет нужными методами, чтобы на нём можно было обучать линейную регрессию (см. семинар 5_sem-sklearn-knn.ipynb):

* Заполните пропуски средними (библиотека SimpleImputer)
* Переведите категориальные признаки в числовые (в методе get_dummies использовать drop_first=True.)
* Разделите датасет на обучающую и тестовую выборку (задать: доля тестовой выборки равна 0.3, `random_state=42`, `shuffle=True`)
* Нормализуйте числовые признаки (при помощи бибилиотеки StandardScaler)

In [ ]:
# -- YOUR CODE HERE --

**Задание 2.3:** Обучите написанную вами линейную регрессию на обучающей выборке

Создаем объект линейной регрессии для `MSELoss`:

In [ ]:
lr_1 = LinearRegression_1(MSELoss()) #создаем регрессор

In [ ]:
# -- YOUR CODE HERE --

**Задание 2.4:** Посчитайте ошибку обученной регрессии на обучающей и тестовой выборке при помощи методов `mean_squared_error` и `r2_score` из `sklearn.metrics`.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# -- YOUR CODE HERE --

Добавим к модели L2 регуляризацию (для борьбы с переобучением). Для этого нам нужно создать новый класс для нового функционала ошибки и его градиента.

Формула функционала ошибки для MSE с L2 регуляризацией выглядит так:
$$
Q(w, X, y) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\langle x_i, w \rangle - y_i)^2 + \lambda ||w||^2
$$

Или в матричном виде:

$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2 + \lambda ||w||^2,
$$

где $\lambda$ — коэффициент регуляризации

Заметим, что (удобно для вычислений):
$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2+\lambda ||w||^2 =\frac{1}{l} (y - Xw)^T(y-Xw)+ \lambda w^Tw.
$$

Градиент $\nabla_w Q(w, X, y)$ выглядит так:

$$
\nabla_w Q(w, X, y) = \frac{2}{\ell} X^T(Xw-y) + 2 \lambda w
$$

**Задание 2.5:** Реализуйте класс `MSEL2Loss`

Он должен вычислять значение функционала ошибки (лосс) $
Q(w, X, y)$ и его градиент $\nabla_w Q(w, X, y)$ по формулам (выше).

Подсказка: обратите внимание, что последний элемент вектора `w` — это bais (сдвиг) (в классе `LinearRegression` к матрице `X` добавляется колонка из единиц — константный признак). bais регуляризовать не нужно. Поэтому не забудьте убрать последний элемент из `w` при подсчёте слагаемого $\lambda||w||^2$ в `calc_loss` и занулить его при подсчёте слагаемого $2 \lambda w$ в `calc_grad`

In [ ]:
class MSEL2Loss(BaseLoss):
    def __init__(self, coef: float = 1.0):
        """
        :param coef: коэффициент регуляризации (лямбда в формуле)
        """
        self.coef = coef

    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета. Последний признак константный.
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии. Последний вес -- bias.
        :output: число -- значения функции потерь
        """    
        # -- YOUR CODE HERE --
        # Вычислите значение функции потерь при помощи X, y и w и верните его

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :output: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        # -- YOUR CODE HERE --
        # Вычислите значение вектора градиента при помощи X, y и w и верните его
        

In [ ]:
#Проверка:
#Создадим объект лосса
losst = MSEL2Loss()

# Создадим датасет
Xt = np.arange(200).reshape(20, 10)
yt = np.arange(20)

# Создадим вектор весов
wt = np.arange(10)

#print(Xt)
#print(yt)
#print(wt)

# Выведем значение лосса и градиента на этом датасете с этим вектором весов
print(losst.calc_loss(Xt, yt, wt))
print(losst.calc_grad(Xt, yt, wt))

# Проверка, что методы реализованы правильно
assert losst.calc_loss(Xt, yt, wt) == 27410487.5, "Метод calc_loss реализован неверно" 
assert np.allclose(
    losst.calc_grad(Xt, yt, wt),
    np.array(
        [
            1163180.0,
            1172283.0,
            1181386.0,
            1190489.0,
            1199592.0,
            1208695.0,
            1217798.0,
            1226901.0,
            1236004.0,
            1245089.0,
        ]
    ),
), "Метод calc_grad реализован неверно"

print("Всё верно!")

Теперь мы можем использовать лосс с l2 регуляризацией в нашей регрессии. Пусть:

In [ ]:
lr_1_l2 = LinearRegression_1(MSEL2Loss(0.1))

**Задание 2.6:** Обучите регрессию с лоссом `MSEL2Loss`. Попробуйте использовать другие коэффициенты регуляризации. Получилось ли улучшить разультат на тестовой выборке? Сравните результат применения регрессии с регуляризацией к обучающей и тестовой выборкам с результатом применения регрессии без регуляризации к тем же выборкам.(Для оценки качества использовать методы `mean_squared_error` и `r2_score` из `sklearn.metrics`).

In [ ]:
# -- YOUR CODE HERE --